In [2]:
# these imports let you use opencv
import cv2 #opencv itself
#import common #some useful opencv functions
#import video # some video stuff
import numpy as np # matrix manipulations
import sys
import math

#the following are to do with this interactive notebook code
%matplotlib inline 
from matplotlib import pyplot as plt # this lets you draw inline pictures in the notebooks
import pylab # this allows you to control figure size 
pylab.rcParams['figure.figsize'] = (10.0, 8.0) # this controls figure size in the notebook
from skimage import data
from skimage.util.dtype import dtype_range
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.morphology import disk
from skimage.morphology import ball
from skimage.filters import rank

C:\Users\madde\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


# Ball Tracking

## Preprocessing

In [13]:
cap = cv2.VideoCapture('eval clip big 2.mp4') # capturing the video in a variable
flag, cap_f = cap.read() # get an initial frame
fbuffer=10
alpha=float(0.9/fbuffer) 
difference_thresh=1
movingaverage=np.float32(cap_f)

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('repport big dilat eval.mp4', fourcc, fps, output_size)

while True:
    ret, frame = cap.read() # capturing the video frame by frame for processing
    if not ret:
        break  # Break the loop if no more frames are available
        
    og_resize = frame[0:1080, 0:1920]
    cv2.accumulateWeighted(frame,movingaverage,alpha)
    res = cv2.convertScaleAbs(movingaverage)
    diff = cv2.absdiff(res, frame) #comparing res to frame

    frame_resize = diff[0:1080, 0:1920]
    # Equalization
    gray = cv2.cvtColor(frame_resize, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(9,9),0)
    ret,th = cv2.threshold(blur,15,255,cv2.THRESH_BINARY)
    dilat = cv2.dilate(th,(7,7),iterations=2)
    # footprint = disk(30)
    # img_local_eq = rank.equalize(th, footprint=footprint)
    # img_glob_eq = exposure.equalize_hist(th)

    # final = cv2.bitwise_and(frame_resize,dilat)


    out.write(dilat)
    cv2.imshow('Video', dilat)
    
    if cv2.waitKey(70) & 0xFF == ord('q'): #Waitkey(num): num sets the delay for processing between each frame
        break  # Press 'q' to exit the loop

# Release the video capture object and close the window
cap.release()
out.release()
cv2.destroyAllWindows()


## Initial LK Tracking

In [138]:
# Load video frames (or use webcam feed)
cap = cv2.VideoCapture("project output.mp4")

# params for corner detection 
feature_params = dict( maxCorners = 80, qualityLevel = 0.7, minDistance = 20, blockSize = 15 ) 
  
# Parameters for lucas kanade optical flow 
lk_params = dict( winSize = (10,10), maxLevel = 20, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 50, 0.9)) 
  
# Create some random colors 
color = np.random.randint(0, 255, (100, 3)) 
  
# Take first frame and find corners in it 
ret, old_frame = cap.read() 
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY) 
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params) 
  
# Create a mask image for drawing purposes 
mask = np.zeros_like(old_frame) 
  
while(1): 
      
    ret, frame = cap.read() 
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
  
    # calculate optical flow 
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params) 
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) 
  
    # Select good points
    good_new = p1[st == 1] 
    good_old = p0[st == 1] 
  
    # draw the tracks 
    for i, (new, old) in enumerate(zip(good_new,  good_old)): 
        a, b = new.ravel().astype(int) 
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2) 
          
        frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1) 
          
    img = cv2.add(frame, mask) 
  
    cv2.imshow('frame', img) 
      
    if cv2.waitKey(50) & 0xFF == ord('q'): #Waitkey(num): num sets the delay for processing between each frame
        break
  
    # Updating Previous frame and points  
    old_gray = frame_gray.copy() 
    p0 = good_new.reshape(-1, 1, 2) 
  
cv2.destroyAllWindows() 
cap.release() 

## LK Tracking P2

In [521]:
import cv2
import numpy as np

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=200,
                      qualityLevel=0.01,
                      minDistance=7,
                      blockSize=7)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Capture the video from a file or camera
cap = cv2.VideoCapture('project output.mp4')

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('Method L-K.mp4', fourcc, fps, output_size)

# Read the first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Find initial points to track
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes

mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
        # frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)

    out.write(img)
    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
out.release()
cap.release()


## LK with Circles (Doesnt Work)

In [560]:
import cv2
import numpy as np

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15,15),
                 maxLevel=8,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

# Function to detect circles using Hough Circle Transform
def detect_circle(frame_gray):
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT, dp=2, minDist=45,param1=58,param2=37,minRadius=15,maxRadius=30)
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        return circles
    return None

# Capture the video from a file or camera
cap = cv2.VideoCapture('project output.mp4')

# Read the first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect circles in the current frame
    current_circle = detect_circle(frame_gray)
    if current_circle is not None:
        p0 = np.array([[current_circle[0][0], current_circle[0][1]]], dtype=np.float32).reshape(-1, 1, 2)
    else:
        print("No circles detected in the current frame.")
        continue  # Skip this frame if no circles are found

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
        frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)

    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\hough.cpp:2269: error: (-215:Assertion failed) !_image.empty() && _image.type() == CV_8UC1 && (_image.isMat() || _image.isUMat()) in function 'cv::HoughCircles'


## LK with Circles P2

In [559]:
import cv2
import numpy as np

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(21, 21),
                 maxLevel=3,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

# Function to detect circles using Hough Circle Transform
def detect_circle(frame_gray):
    circles = cv2.HoughCircles(frame_gray,cv2.HOUGH_GRADIENT, dp=2, minDist=45,param1=58,param2=37,minRadius=10,maxRadius=58)
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        return circles
    return None

# Capture the video from a file or camera
cap = cv2.VideoCapture('project output.mp4')

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('Method L-K with Circles.mp4', fourcc, fps, output_size)

# Read the first frame
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect circles in the current frame
    current_circle = detect_circle(frame_gray)
    if current_circle is not None:
        p0 = np.array([[current_circle[0][0], current_circle[0][1]]], dtype=np.float32).reshape(-1, 1, 2)
    else:
        # If no circles are found, continue to the next frame
        continue

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
        frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

    img = cv2.add(frame, mask)

    out.write(img)
    cv2.imshow('frame', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
out.release()
cap.release()


## Hough Circle only

### Smaller Circles

Better for gameplay 

In [15]:
cap = cv2.VideoCapture('repport big dilat eval.mp4')
cap2 = cv2.VideoCapture('eval clip big 2.mp4')

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('repport big circle.mp4', fourcc, fps, output_size)

prevCircle = None
dist = lambda x1,y1,x2,y2: (x1-x2)**2 + (y1-y2)**2

while True:
    ret2,frame2 = cap2.read()
    ret1,frame = cap.read()
    
    if not ret1: break
    if not ret2: break

    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurFrame = cv2.GaussianBlur(grayFrame, (21,21),0)

    circles = cv2.HoughCircles(grayFrame ,cv2.HOUGH_GRADIENT, dp=1.4, minDist=100,
                               param1=80,param2=17,minRadius=5,maxRadius=20)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        chosen = None
        for i in circles[0, :]:
            if chosen is None: chosen = i
            if prevCircle is not None:
                if dist(chosen[0], chosen[1],prevCircle[0],prevCircle[1]) <= dist(i[0],i[1],prevCircle[0],prevCircle[1]):
                    chosen = i
        cv2.circle(frame2, (chosen[0], chosen[1]), 1, (0,100,100), 3)
        cv2.circle(frame2, (chosen[0], chosen[1]), chosen[2], (255,0,255), 3)
        prevCircle = chosen

    out.write(frame2)
    cv2.imshow("circles", frame2)

    if cv2.waitKey(70) & 0xFF == ord('q'): break


cv2.destroyAllWindows()
out.release()
cap.release()

C:\Users\madde\AppData\Local\Temp\ipykernel_31892\1138968047.py:12: RuntimeWarning: overflow encountered in scalar subtract
  dist = lambda x1,y1,x2,y2: (x1-x2)**2 + (y1-y2)**2


### Big Circles

Footage more zoomed in requiring larger circle

In [540]:
cap = cv2.VideoCapture('test output.mp4')
cap2 = cv2.VideoCapture('Left2Right.MOV')

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('Result L2R.mp4', fourcc, fps, output_size)

length = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)
prevCircle = None
dist = lambda x1,y1,x2,y2: (x1-x2)**2 + (y1-y2)**2

while True:
    ret,frame = cap.read()
    ret2,frame2 = cap2.read()
    if not ret: break
    if not ret2: break

    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurFrame = cv2.GaussianBlur(grayFrame, (21,21),0)

    circles = cv2.HoughCircles(grayFrame ,cv2.HOUGH_GRADIENT, dp=1.4, minDist=100,
                               param1=100,param2=25,minRadius=12,maxRadius=58)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        chosen = None
        for i in circles[0, :]:
            if chosen is None: chosen = i
            if prevCircle is not None:
                if dist(chosen[0], chosen[1],prevCircle[0],prevCircle[1]) <= dist(i[0],i[1],prevCircle[0],prevCircle[1]):
                    chosen = i
        cv2.circle(frame2, (chosen[0], chosen[1]), 1, (0,100,100), 3)
        cv2.circle(frame2, (chosen[0], chosen[1]), chosen[2], (255,0,255), 3)
        prevCircle = chosen
        
    cv2.namedWindow("circles", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("circles", 2000, 1500)
    out.write(frame2)
    cv2.imshow("circles", frame2)
    

    if cv2.waitKey(10) & 0xFF == ord('q'): break


cv2.destroyAllWindows()
out.release()
cap.release()

248


C:\Users\madde\AppData\Local\Temp\ipykernel_23032\68478830.py:14: RuntimeWarning: overflow encountered in scalar subtract
  dist = lambda x1,y1,x2,y2: (x1-x2)**2 + (y1-y2)**2


# Line Detection

In [544]:
def main(argv):
    default_file = 'CV clip.mp4'  # This is not useful
    filename = argv[0] if len(argv) > 0 else default_file

    # Open the video file
    cap = cv2.VideoCapture('CV input.mp4')

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
    fps = 60.0  # Frames per second
    width, height = 1920, 1080  # Frame size
    output_size = (width, height)
    out = cv2.VideoWriter('Results Lines.mp4', fourcc, fps, output_size)
    
    if not cap.isOpened():
        print('Error opening video file!')
        return -1

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Convert frame to grayscale
        r_frame = frame[0:1080, 0:1920]
        src = cv2.cvtColor(r_frame, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection
        dst = cv2.Canny(src, 200, 255, None, 3)

        # Copy edges to the images that will display the results in BGR
        cdst = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
        cdstP = np.copy(cdst)

        # Hough line detection
        lines = cv2.HoughLines(dst, 1, np.pi / 80, 200, None, 0, 0)
        if lines is not None:
            for i in range(len(lines)):
                rho = lines[i][0][0]
                theta = lines[i][0][1]
                a = math.cos(theta)
                b = math.sin(theta)
                x0 = a * rho
                y0 = b * rho
                pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * (a)))
                pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * (a)))
                cv2.line(cdst, pt1, pt2, (0, 0, 255), 3, cv2.LINE_AA)

        # Hough line detection (probabilistic)
        linesP = cv2.HoughLinesP(dst, 0.4, np.pi / 80, 200, None, 180, 62)
        if linesP is not None:
            for i in range(len(linesP)):
                l = linesP[i][0]
                # cv2.line(frame, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)

        # Display results

        out.write(frame)
        cv2.imshow('Fully lined', frame) #correct length lines

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return 0

if __name__ == "__main__":
    main(sys.argv[1:])


# Merging Circles and Lines

In [558]:
cap = cv2.VideoCapture('Demo Pres.mp4')
cap2 = cv2.VideoCapture('CV input.mp4')

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('Demo final.mp4', fourcc, fps, output_size)

length = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
print(length)
prevCircle = None
dist = lambda x1,y1,x2,y2: (x1-x2)**2 + (y1-y2)**2

while True:
    ret2,frame2 = cap2.read() #original footage
    ret1,frame = cap.read() #processed footage for circle tracking
    
    if not ret1: break
    if not ret2: break

    grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    src = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    blurFrame = cv2.GaussianBlur(grayFrame, (21,21),0)

    circles = cv2.HoughCircles(grayFrame ,cv2.HOUGH_GRADIENT, dp=1.4, minDist=100,
                               param1=70,param2=17,minRadius=5,maxRadius=20)
    dst = cv2.Canny(src, 200, 255, None, 3)
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        chosen = None
        for i in circles[0, :]:
            if chosen is None: chosen = i
            if prevCircle is not None:
                if dist(chosen[0], chosen[1],prevCircle[0],prevCircle[1]) <= dist(i[0],i[1],prevCircle[0],prevCircle[1]):
                    chosen = i
        cv2.circle(frame2, (chosen[0], chosen[1]), 1, (0,100,100), 3)
        cv2.circle(frame2, (chosen[0], chosen[1]), chosen[2], (255,0,255), 3)
        prevCircle = chosen

    # Hough line detection (probabilistic)
    linesP = cv2.HoughLinesP(dst, 0.4, np.pi / 80, 200, None, 180, 100)
    if linesP is not None:
        for i in range(len(linesP)):
            l = linesP[i][0]
            cv2.line(frame2, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)
       
    # cv2.namedWindow("circles", cv2.WINDOW_NORMAL)
    # cv2.resizeWindow("circles", 2000, 1500)
    out.write(frame2)
    cv2.imshow("circles", frame2)
    
    if cv2.waitKey(5) & 0xFF == ord('q'): break
    
cv2.destroyAllWindows()
out.release()
cap.release()
cap2.release()

1377


C:\Users\madde\AppData\Local\Temp\ipykernel_23032\3142273687.py:14: RuntimeWarning: overflow encountered in scalar subtract
  dist = lambda x1,y1,x2,y2: (x1-x2)**2 + (y1-y2)**2


Polygon of inside the cour

In [547]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    # Retrieve the number of color channels of the image.
    channel_count = img.shape[2]
    # Create a match color with the same color channel counts.
    match_mask_color = (255,) * channel_count
      
    # Fill inside the polygon
    cv2.fillPoly(mask, vertices, match_mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

region_of_interest_vertices = [
    (1550, 800),
    (1170,582),
    (783,573),
    (368,800),
]

image = cv2.imread('Capture1.PNG')
cap2 = cv2.VideoCapture('CV input.mp4')

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Codec for AVI format
fps = 60.0  # Frames per second
width, height = 1920, 1080  # Frame size
output_size = (width, height)
out = cv2.VideoWriter('Method Polygon.mp4', fourcc, fps, output_size)

while True:
    ret,frame = cap2.read()
    if not ret: break
        
    cropped_image = region_of_interest(frame,np.array([region_of_interest_vertices], np.int32),)

    out.write(cropped_image)
    cv2.imshow('frame',cropped_image)
    if cv2.waitKey(10) & 0xFF == ord('q'): break

cv2.destroyAllWindows()
out.release()
cap2.release()    